In [61]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
import torch
import os

In [62]:
from sklearn.datasets import fetch_openml

if os.path.exists('X.npy') and os.path.exists('y.npy'):
    X = np.load('X.npy', allow_pickle=True)
    y = np.load('y.npy', allow_pickle=True)
else:
    X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
    np.save('X.npy', X)
    np.save('y.npy', y)

In [63]:
# X, y = load_digits(return_X_y=True)

y = np.eye(10)[y.astype(np.int32)].astype(np.float32)
X = StandardScaler().fit_transform(X)

In [64]:
X.shape, y.shape

((70000, 784), (70000, 10))

In [65]:
num_features = 784

In [66]:
# X = X[:100, :num_features]
# y = y[:100]

In [67]:
w1 = torch.repeat_interleave(torch.linspace(-0.1, 0.1, 32)[None, ...], num_features, dim=0)
w2 = torch.repeat_interleave(torch.linspace(-0.1, 0.1, 10)[None, ...], 32, dim=0)

# w1 = torch.randn(73, 32) * 0.01
# w2 = torch.randn(32, 10) * 0.01

b1 = torch.zeros(32)
b2 = torch.zeros(10)

X_t = torch.from_numpy(X).float()
y_t = torch.from_numpy(y).float()

In [68]:
batch_size = 128

In [69]:
X.shape, y.shape

((70000, 784), (70000, 10))

In [70]:
lr = 0.01

In [71]:
for i in range(1):
    for j in range(0, 5 * batch_size, batch_size):
        
        X_batch = X_t[j: j + batch_size]
        y_batch = y_t[j: j + batch_size]


        w1.requires_grad = True
        b1.requires_grad = True
        w2.requires_grad = True
        b2.requires_grad = True


        z1 = X_batch @ w1 + b1
    #     print(torch.norm(z1))
        a1 = torch.max(torch.zeros_like(z1), z1)
    #     a1 = torch.nn.functional.relu(z1)
    #     print(torch.norm(a1))

        z2 = a1 @ w2 + b2
        sm_scale = z2 - torch.max(z2, axis=-1, keepdims=True)[0]
        y__t = torch.exp(sm_scale) / torch.sum(torch.exp(sm_scale), keepdims=True, axis=-1)
        assert y__t.shape == y_batch.shape
    #     print(torch.norm(y__t))
        logy = torch.log(y__t)
    #     logy.requires_grad = True

    #     print(torch.norm(logy))
        ce = - y_batch * logy
        l = torch.sum(ce) / len(y_batch)
    #     l = torch.sum(ce)
        l.backward()
        print(torch.norm(w1.grad))
        print(torch.norm(w2.grad))

        if i % 1 == 0:
            print(l)

        w1 = w1.data - lr * w1.grad
        w2 = w2.data - lr * w2.grad

        b1 = b1.data - lr * b1.grad
        b2 = b2.data - lr * b2.grad


    
    

tensor(2.3828)
tensor(13.6359)
tensor(10.1083, grad_fn=<DivBackward0>)
tensor(1.5576)
tensor(7.7059)
tensor(6.4972, grad_fn=<DivBackward0>)
tensor(1.4608)
tensor(8.2745)
tensor(6.0974, grad_fn=<DivBackward0>)
tensor(1.7375)
tensor(9.9127)
tensor(6.8717, grad_fn=<DivBackward0>)
tensor(1.2178)
tensor(6.7767)
tensor(4.7786, grad_fn=<DivBackward0>)


In [72]:
class Linear:
    def __init__(self, in_dim, out_dim):
        self.w = np.zeros([in_dim, out_dim])
        self.w = np.repeat(np.linspace(-0.1, 0.1, out_dim)[np.newaxis, ...], in_dim, axis=0)

#         self.w = np.random.randn(in_dim, out_dim)
        self.b = np.zeros([1, out_dim])
        self.dw = None
        self.db = None
        self.in_dim = self.w.shape[0]
        self.out_dim = self.w.shape[1]

        
    def forward(self, x):
        self.x = x
        return np.matmul(x, self.w) + self.b
    
    def backward(self, d):
        self.db = np.mean(d, axis=0)
        assert self.db.shape == self.b.shape, (d.shape, self.db.shape, self.b.shape)
        
        J = np.zeros([self.x.shape[0], self.out_dim, np.prod(self.w.shape)])
        j = 0
        for i in range(self.out_dim):
            J[:, i: i + 1, j: j + self.in_dim] = self.x[:, np.newaxis, :]
            j += self.in_dim
        
        dw = d @ J
        
        dw = np.reshape(np.mean(dw, axis=0), self.w.shape, order='F')
        
        self.dw = dw
        
        d = d @ np.repeat(self.w.T[np.newaxis, ...], d.shape[0], axis=0)
        
        return d
        
    def step(self, lr):
        self.w = self.w - lr * self.dw
        self.b = self.b - lr * self.db

In [73]:
class ReLU:
    def __init__(self):
        self.a = None
        
    def forward(self, x):
        self.a = np.maximum(x, 0)
        return self.a
    
    def backward(self, d):
        return d * (self.a != 0)[:, np.newaxis, :].astype(np.float32)
        

In [74]:
class Softmax:
    def __init__(self):
        self.a = None
        
    def forward(self, x):
        assert len(x.shape) == 2
        x = x - np.max(x, axis=-1, keepdims=True)
        self.a = np.exp(x) / np.sum(np.exp(x), keepdims=True, axis=-1)
        return self.a
    def backward(self, d):
        
        diag = np.stack([np.diag(self.a[i]) for i in range(len(self.a))])
        op = np.stack([np.outer(self.a[i], self.a[i]) for i in range(len(self.a))])
        J = diag - op
        
        return d[:, np.newaxis, ...] @ J

In [75]:
class CrossEntropy:
    def forward(self, y_, y):
        
        l = - np.sum(y * np.log(y_))
        l /= len(y)
        return y_, l
    
    def backward(self, y_, y):
        assert y_.shape == y.shape
        d =  - y / y_
        return d

In [76]:
class MLP:
    def __init__(self):
        self.linear1 = Linear(784, 32)
        self.relu1 = ReLU()
        self.linear2 = Linear(32, 10)
#         self.relu2 = ReLU()
#         self.linear3 = Linear(32, 10)
        self.softmax = Softmax()
        self.loss = CrossEntropy()
        
    def forward(self, x, y):
        x = self.linear1.forward(x)
        x = self.relu1.forward(x)
        x = self.linear2.forward(x)
#         x = self.relu2.forward(x)
#         x = self.linear3.forward(x)
        x = self.softmax.forward(x)
        loss = self.loss.forward(x, y)
        return loss
    
    def backward(self, y_, y):
        d = self.loss.backward(y_, y)
        d = self.softmax.backward(d)
#         d = self.linear3.backward(d)
#         d = self.relu2.backward(d)
        d = self.linear2.backward(d)
        d = self.relu1.backward(d)
        d = self.linear1.backward(d)

    
    def step(self, lr):
        print(np.linalg.norm(self.linear1.dw))
        self.linear1.step(lr)
        print(np.linalg.norm(self.linear2.dw))
        self.linear2.step(lr)
#         self.linear3.step(lr)

        

In [78]:
mlp = MLP()

In [79]:
for i in range(1):
#     for j in range(0, len(X), batch_size):
    for j in range(0, 5 * batch_size, batch_size):

        X_batch = X[j: j + batch_size]
        y_batch = y[j: j + batch_size]
        y_, loss = mlp.forward(X_batch, y_batch)
        mlp.backward(y_, y_batch)
        mlp.step(lr=0.01)
        if j % 1 == 0:
#             print(j)
            print(loss, 'loss')
            print((np.argmax(y_, axis=-1) == np.argmax(y_batch, axis=-1)).sum() / len(y_batch))

2.3828043360516906
13.635874886491942
10.10825352110885 loss
0.1015625
1.557595194972053
7.705902286545122
6.497165056695555 loss
0.125
1.4608140989410092
8.274531718494313
6.097363457068974 loss
0.1640625
1.7374628317471967
9.912711029512327
6.871654922894726 loss
0.1484375
1.2178036918121307
6.776748023687062
4.778580232578671 loss
0.09375
